# Fig3-Analysis of Ecdysone Receptor nuclear concentration

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
from glob import glob
from scipy.stats import zscore
from matplotlib import rc
import matplotlib as mpl

/home/brandon/anaconda3/envs/ome_zarr/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
%matplotlib qt

In [37]:
rc('axes', linewidth=4)
linewidth=4
fontsize=24
mpl.rc('axes', linewidth=linewidth)
mpl.rc('font', family='Arial')

colors = ((27,158,119),
          (217,95,2),
          (117,112,179),
          (231,41,138),
          (102,166,30),
          (230,171,2))

def style_axes(ax, fontsize=24):
    plt.minorticks_off()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    plt.tight_layout()
    
    return ax

In [4]:
def bin_aps(df, bins, ap_col='y'):
    _counts, bins = np.histogram(df.get(ap_col), bins)
    bins = bins[1:]
    partial_func = partial(get_ap_bin, bins=bins)
    binned_aps = df.get(ap_col).apply(partial_func)
    df[f'binned_{ap_col}'] = binned_aps.values
    
    return df
    

def get_ap_bin(this_ap, bins):
    this_bin = np.where(np.abs(this_ap - bins) == np.nanmin(np.abs(this_ap - bins)))[0][0]

    return this_bin
    



In [5]:
experiment_paths = [
    r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_02_06_ca-Gal4_UAS-His-RFP_mNG-EcR-B1']

quant_col = 'ch0'
method = 'bkg_sub_mean_ch0'
#method = 'raw_mean_ch0'

df_name = 'nuclei_3_quant_v4_bkg_by_slice.pkl'

In [6]:
larvae_dir = r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_02_06_ca-Gal4_UAS-His-RFP_mNG-EcR-B1/larva_2'
df = pd.read_pickle(larvae_dir + '/' + df_name)

In [7]:
df.y.max()

1391

In [8]:
ys = df.y
counts, bins = np.histogram(ys, bins=50)
plt.figure()
plt.bar(bins[:-1], counts, width=10)

<BarContainer object of 50 artists>

In [9]:
counts

array([10, 22, 26, 30, 28, 24, 20,  7,  3,  3,  8, 10, 14, 21, 32, 35, 27,
       25, 28, 32, 39, 40, 28, 27, 32, 29, 31, 38, 34, 27, 19, 36, 38, 33,
       20, 19, 26, 32, 38, 26, 20, 23, 34, 26, 17, 12,  9, 13,  3,  2])

In [16]:
n_bins = 22
all_means = np.zeros((6, n_bins))
all_stds = np.zeros((6, n_bins))
bins = np.linspace(0, 1, n_bins + 1)
ap_col = 'ap'
counter= 0
q95 = np.zeros(6)
plt.figure(figsize=(12, 6))
for path in experiment_paths:
    larvae_dirs = glob(path + '/larva*')
    for larvae_dir in larvae_dirs:
        if larvae_dir == r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_02_06_ca-Gal4_UAS-His-RFP_mNG-EcR-B1/larva_1':
            continue
        df = pd.read_pickle(larvae_dir + '/' + df_name)

    
        df['ap'] = df.y / df.y.max()
        df = bin_aps(df, bins, ap_col='ap')

        binned_means = df.get([method, f'binned_{ap_col}']).groupby(by=f'binned_{ap_col}').mean().values.flatten()
        #binned_means = df.get([method, f'binned_{ap_col}']).groupby(by=f'binned_{ap_col}').sum().values.flatten()

        binned_std = df.get([method, f'binned_{ap_col}']).groupby(by=f'binned_{ap_col}').std().values.flatten()
        
        m = binned_means
        s = binned_std
        l = m -s
        u = m + s
        ap = bins[:-1] / np.max(bins[:-1])
        plt.plot(ap, m, color=np.array(colors[counter]) / 255, linewidth=4)
        plt.plot(ap, l, '--', color=np.array(colors[counter]) / 255, linewidth=2, alpha=0.5)
        plt.plot(ap, u, '--', color=np.array(colors[counter]) / 255, linewidth=2, alpha=0.5)

        all_means[counter] = binned_means
        all_stds[counter] = binned_std
        q95[counter] = np.quantile(df.get(method), 0.95)

        counter += 1
        
plt.xlabel('fraction of anterior-posterior axis', fontsize=fontsize)
plt.ylabel('mean EcR-B1 \nfluorescence intensity (a.u.)', fontsize=fontsize)
plt.xlim([0, 1])
style_axes(plt.gca())






<AxesSubplot:xlabel='fraction of anterior-posterior axis', ylabel='mean EcR-B1 \nfluorescence intensity (a.u.)'>

In [39]:
color=(0, 150 / 255, 150 / 255)

plt.figure(figsize=(13, 5.5))

mean_q95 = np.mean(q95)
m = np.mean(all_means, axis=0)
#s = np.std(all_means, axis=0)
# uncertainty is dominated by individual uncertainties
s = np.sqrt(np.mean(all_stds ** 2, axis=0) / len(all_stds))

l = m -s
u = m + s
plt.plot(bins[:-1] / np.max(bins[:-1]), m, color=color, linewidth=4, label='mean')
plt.fill_between(bins[:-1] / np.max(bins[:-1]), l, u, facecolor=color, alpha=0.3, label='_nolabel_')
plt.xlabel('fraction of anterior-posterior axis', fontsize=fontsize)
plt.ylabel('mean mNG-EcR-B1 \nfluorescence intensity (a.u.)', fontsize=fontsize)
plt.xlim([0, 1])
style_axes(plt.gca())


<AxesSubplot:xlabel='fraction of anterior-posterior axis', ylabel='mean mNG-EcR-B1 \nfluorescence intensity (a.u.)'>

In [40]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig3-Ecr/ecr_ap_dist_shaded_errorbar.pdf')

In [21]:
n_cells = df.get([method, f'binned_{ap_col}']).groupby(by=f'binned_{ap_col}').size().values.flatten()

In [24]:
plt.figure()
plt.plot(bins[:-1], n_cells)

In [25]:
np.sum(n_cells)

577

In [38]:
plt.figure(figsize=(12, 6))
n_cell_arr = np.zeros((6, len(bins) - 1))
counter = 0
for path in experiment_paths:
    larvae_dirs = glob(path + '/larva*')
    for larvae_dir in larvae_dirs:
        if larvae_dir == r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_02_06_ca-Gal4_UAS-His-RFP_mNG-EcR-B1/larva_1':
            continue
        df = pd.read_pickle(larvae_dir + '/' + df_name)

    
        df['ap'] = df.y / df.y.max()
        df = bin_aps(df, bins, ap_col='ap')

        n_cells = df.get([method, f'binned_{ap_col}']).groupby(by=f'binned_{ap_col}').size().values.flatten()
        plt.plot(bins[:-1], n_cells)
        print(np.sum(n_cells))
        n_cell_arr[counter] = n_cells
        counter += 1

1176
933
754
818
919
577


In [28]:
np.mean([1176, 933, 754, 818, 919, 577])

862.8333333333334

In [29]:
np.std([1176, 933, 754, 818, 919, 577])

183.35159000007488

In [30]:
863 / 2200

0.3922727272727273

In [33]:
plt.figure()
plt.plot(bins[:-1], np.mean(n_cell_arr, axis=0))

In [35]:
mean_n_cell_arr = np.mean(n_cell_arr, axis=0)

In [36]:
mean_n_cell_arr

array([17.5       , 38.        , 40.83333333, 34.16666667, 30.33333333,
       35.66666667, 49.66666667, 51.83333333, 50.66666667, 49.83333333,
       51.33333333, 50.83333333, 46.16666667, 48.83333333, 43.33333333,
       42.83333333, 43.5       , 38.66666667, 39.16666667, 31.        ,
       21.66666667,  7.        ])